In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
output_folder_csv = "../output/csv"
video_name = "release_test_01"

In [ ]:
df_raw_angles = pd.read_csv(os.path.join(output_folder_csv, f"{video_name}.csv"))
df_raw_angles

In [ ]:
def replace_extremes_with_average(df, column, threshold):
    result_df = df.copy()

    # Find extreme values
    extremes = result_df[column][np.abs((result_df[column].diff())) > threshold | (result_df[column].diff(-1) > threshold)]

    # Replace extreme values with the average of the values at positions -1 and +1
    for idx in extremes.index:
        if idx - 1 >= 0 and idx + 1 < len(result_df):
            result_df.at[idx, column] = (result_df.at[idx - 1, column] + result_df.at[idx + 1, column]) / 2

    return result_df

# # Example usage
# data = {'Values': [140, 140, 1450, 139, 200, 139, 140]}
# df = pd.DataFrame(data)
#
# window_size = 3  # Adjust as needed
# threshold = 100   # Adjust as needed
# column_to_process = 'Values'
#
# result_df = replace_extremes_with_average(df, c

# Angle / Time plot

In [ ]:
# Create a Line Graph using Plotly Express
fig = px.line(df_raw_angles, x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Angles per time')
# Show the plot
fig.show()

## Filter extremas

In [ ]:

df_raw_angles = replace_extremes_with_average(df_raw_angles,"angle_0",50)
df_raw_angles = replace_extremes_with_average(df_raw_angles,"angle_1",50)
df_raw_angles = replace_extremes_with_average(df_raw_angles,"angle_2",50)

# Create a Line Graph using Plotly Express
fig = px.line(df_raw_angles, x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Angles per time')
# Show the plot
fig.show()

## Smoothing demonstration

In [ ]:
window_size = 30
fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Angles per time (smooth)')
# Show the plot
fig.show()

# Calculate angular speed
Angular speed of a rotating object: An angular speed of a rotating object is the amount of the angle changed per unit time.

I use windows averaging for smoothness

In [ ]:
# Calculate delta_time and angular speeds
window_size = 30
delta_time = np.diff(df_raw_angles['time'])
angular_0_speed = np.diff(df_raw_angles['angle_0'].rolling(window=window_size).mean())
angular_1_speed = np.diff(df_raw_angles['angle_1'].rolling(window=window_size).mean())
angular_2_speed = np.diff(df_raw_angles['angle_2'].rolling(window=window_size).mean())

delta_time = np.insert(delta_time, 0, np.NAN)
angular_0_speed = np.insert(angular_0_speed, 0, np.NAN)
angular_1_speed = np.insert(angular_1_speed, 0, np.NAN)
angular_2_speed = np.insert(angular_2_speed, 0, np.NAN)


df_raw_angles["delta_time"] = delta_time
df_raw_angles["angular_0_speed"] = angular_0_speed
df_raw_angles["angular_1_speed"] = angular_1_speed
df_raw_angles["angular_2_speed"] = angular_2_speed

In [ ]:
# Create a Line Graph using Plotly Express

fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
# Show the plot
fig.show()

## Calcualte average speed

In [ ]:
window = 10
df_raw_angles['average_speed'] = np.mean(df_raw_angles[["angular_0_speed","angular_1_speed","angular_2_speed"]].rolling(window=window_size).mean(), axis=1)

In [ ]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles.rolling(window=window_size).mean(), x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed',],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].rolling(window=window).mean(),
                         mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

# Is finger moving?

In [ ]:
threshold = 0.2
# df_raw_angles["is_move"] = (np.abs(df_raw_angles['average_speed'].rolling(window=window).mean())  > threshold) | (np.abs(df_raw_angles['angular_0_speed'].rolling(window=window).mean())  > threshold)
# df_raw_angles["is_move"] = np.abs(df_raw_angles['average_speed'].rolling(window=window).mean())  > threshold
window = 10
angle_columns = ["angle_0", "angle_1", "angle_2"]
speed_columns = ['angular_0_speed', 'angular_1_speed', 'angular_2_speed']
# df_raw_angles["is_move"] = np.any(np.abs(df_raw_angles[['angular_0_speed', 'angular_1_speed', 'angular_2_speed']].rolling(window=window).mean())  > threshold,axis=1)

df_raw_angles["is_move"] = np.abs(df_raw_angles["average_speed"]) > threshold

In [ ]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles, x='time', y=['angular_0_speed', 'angular_1_speed', 'angular_2_speed'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"],
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"] * -1,
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))

fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'],
                         mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

In [ ]:
# Create a Line Graph using Plotly Express
window = 10
fig = px.line(df_raw_angles, x='time', y=['angle_0', 'angle_1', 'angle_2'],  labels={'value': 'Values'},title='Speed / Time (smooth)')
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].max() * df_raw_angles["is_move"] * 40 ,
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Is move', showlegend=False))

# fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'].rolling(window=window).mean(),
#                          mode='lines', name=f'Average Speed', line=dict(color='black')))
# Show the plot
fig.show()

# Which way finger moving?

In [ ]:
# Create masks for positive and negative 'move' regions
# df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & (df_raw_angles["average_speed"] > 0.0)
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (df_raw_angles["average_speed"] < -0.0)

window= 30

# df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() > 0.0, axis=1))
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() < 0.0, axis=1))

df_raw_angles["positive_move_mask"] = (df_raw_angles["is_move"]) & np.all(
    np.array([df_raw_angles["angular_0_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0,
              df_raw_angles["angular_1_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0,
              df_raw_angles["angular_2_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) > 0] ),axis=0)
# df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & (np.all(df_raw_angles[speed_columns].rolling(window=window).mean() < 0.0, axis=1))
df_raw_angles["negative_move_mask"] = (df_raw_angles["is_move"]) & np.all(
    np.array([df_raw_angles["angular_0_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0,
              df_raw_angles["angular_1_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0,
              df_raw_angles["angular_2_speed"].rolling(window=window).mean().rolling(window,center = True).apply(lambda x: np.mean(sorted(x)[1:window-1])) < 0] ),axis=0)


In [ ]:
# Create a Line Graph using Plotly Express
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, shared_yaxes=True)

window= 30
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)

for speed_column in speed_columns:
    # Plot smoothed angle data
    fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles[speed_column].rolling(window=window).mean(),
                             mode='lines', name=f'{speed_column}'))

# Add shaded regions for positive 'move' intervals
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=np.mean(df_raw_angles[speed_columns],axis=1).max() * df_raw_angles["positive_move_mask"],
                         fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Positive Move Region - {speed_columns}', showlegend=False))

# Add shaded regions for negative 'move' intervals
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=np.mean(df_raw_angles[speed_columns],axis=1).max()* df_raw_angles["negative_move_mask"] * -1,
                         fill='toself', fillcolor='rgba(0, 0, 255, 0.2)',
                         line=dict(color='rgba(255, 255, 255, 0)'),
                         name=f'Negative Move Region - {speed_columns}', showlegend=False))

# Plot the average speed
fig.add_trace(go.Scatter(x=df_raw_angles['time'], y=df_raw_angles['average_speed'],
                         mode='lines', name=f'Average Speed', line=dict(color='black')))

# Update layout
fig.update_layout(title_text=f'Angles per frame with Move Regions and Average Speed')

# Show the plot
fig.show()


In [629]:
colors.append("black")

In [630]:
loop_intervals = [(7.36, 7.6, 7.92),
                  (8.10, 8.32, 8.66),
                  (8.8, 9.04, 9.35),
                  (9.48, 9.7, 10.05),
                  (10.12,10.36,10.69)]

In [631]:
window= 20
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)
fig = go.Figure()
for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[0]) & (df_raw_angles["time"] < interval[2])]
    # for speed_column in speed_columns:
    #     # Plot smoothed angle data
    #     fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0], y=df_time_lookup[speed_column].rolling(window=window).mean(),
    #                              mode='lines', name=f'{speed_column}'))
    # Plot the average speed
    fig.add_trace(go.Scatter(x=df_time_lookup['time']-interval[0], y=df_time_lookup['average_speed'],
                             mode='lines', name=f'ABG Speed measure {measure_idx}', line=dict(color=colors[measure_idx])))


fig.show()

In [632]:
window= 20
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)
fig = go.Figure()
angle_agregator = []
for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[0]) & (df_raw_angles["time"] < interval[2])]
    # for speed_column in angle_columns:
    speed_column = angle_columns[0]
    # Plot smoothed angle data
    fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0], y=df_time_lookup[speed_column].rolling(window=window).mean(),
                             mode='lines', name=f'Measure {measure_idx} - {speed_column}'))
    # Plot the average speed
    # fig.add_trace(go.Scatter(x=df_time_lookup['time']-interval[0], y=df_time_lookup['average_speed'],
    #                          mode='lines', name=f'Average Speed', line=dict(color='black')))
    angle_agregator.append(list(df_time_lookup[speed_column]))

end_index = min([len(agre) for agre in angle_agregator])
a = [np.array(agre[0:end_index]) for agre in angle_agregator]
y_average = np.mean(np.array(a),axis=0)
df_local = pd.DataFrame(data=y_average, columns=["average"])

fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0] , y=df_local["average"].rolling(window=window).mean(),
                         mode='lines', name=f'Measure {measure_idx} - {speed_column}', line=dict(color=colors[measure_idx])),)


fig.show()

In [ ]:
window= 20
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)
fig = go.Figure()
angle_agregator = []
for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[0]) & (df_raw_angles["time"] < interval[2])]
    # for speed_column in angle_columns:
    speed_column = angle_columns[1]
    # Plot smoothed angle data
    fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0], y=df_time_lookup[speed_column].rolling(window=window).mean(),
                             mode='lines', name=f'Measure {measure_idx} - {speed_column}'))
    # # Plot the average speed
    # fig.add_trace(go.Scatter(x=df_time_lookup['time']-interval[0], y=df_time_lookup['average_speed'],
    #                          mode='lines', name=f'Average Speed', line=dict(color='black')))
    angle_agregator.append(list(df_time_lookup[speed_column]))

end_index = min([len(agre) for agre in angle_agregator])
a = [np.array(agre[0:end_index]) for agre in angle_agregator]
y_average = np.mean(np.array(a),axis=0)
df_local = pd.DataFrame(data=y_average, columns=["average"])

fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0] , y=df_local["average"].rolling(window=window).mean(),
                         mode='lines', name=f'Measure {measure_idx} - {speed_column}', line=dict(color=colors[measure_idx])),)

fig.show()

In [ ]:
window= 20
# # Calculate the average speed of the three angles
# df_angle_speed["average_speed"] = df_raw_angles[speed_columns].mean(axis=1)
fig = go.Figure()
angle_agregator = []

for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[0]) & (df_raw_angles["time"] < interval[2])]
    # for speed_column in angle_columns:
    speed_column = angle_columns[2]
    # Plot smoothed angle data
    fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0], y=df_time_lookup[speed_column].rolling(window=window).mean(),
                             mode='lines', name=f'Measure {measure_idx} - {speed_column}'))
    # Plot the average speed
    # fig.add_trace(go.Scatter(x=df_time_lookup['time']-interval[0], y=df_time_lookup['average_speed'],
    #                          mode='lines', name=f'Average Speed', line=dict(color='black')))
    angle_agregator.append(list(df_time_lookup[speed_column]))

end_index = min([len(agre) for agre in angle_agregator])
a = [np.array(agre[0:end_index]) for agre in angle_agregator]
y_average = np.mean(np.array(a),axis=0)
df_local = pd.DataFrame(data=y_average, columns=["average"])

fig.add_trace(go.Scatter(x=df_time_lookup['time']- interval[0] , y=df_local["average"].rolling(window=window).mean(),
                         mode='lines', name=f'Measure {measure_idx} - {speed_column}', line=dict(color=colors[measure_idx])),)

fig.show()

In [ ]:
list(df_time_lookup['time'][:end_index])

In [ ]:
# Create average plot per angle

In [ ]:
a = [np.array(agre[0:end_index]) for agre in angle_agregator]

In [ ]:
np.mean(np.array(a),axis=0)

In [ ]:
df_raw_angles

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d
fig = go.Figure()
# Create a curve using numpy's polyfit function
degree = 1  # You can adjust the degree of the polynomial
trajectory_points_name = [("i_p_0_x", "i_p_0_y"), ("i_p_1_x", "i_p_1_y"), ("i_p_2_x", "i_p_2_y"), ("i_p_3_x", "i_p_3_y")]
fig = make_subplots(rows=2, cols=1, subplot_titles=['First Interval', 'Second Interval'])
colors = ["blue", "pink", "green", "orange"]

agregated_points_down_x = []
agregated_points_down_y = []
for measure_idx, interva in enumerate(loop_intervals):
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] >interva[0]) & (df_raw_angles["time"] < interva[1])]

    agregated_points_down_measure_x = []
    agregated_points_down_measure_y = []
    print(interva[2]-interva[0])

    for index in range(0, len(trajectory_points_name)):
        # pp = trajectory_points_name[index]
        pp = trajectory_points_name[index]

        unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]

        # Create a cubic spline interpolation
        x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
        y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1
        spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

        # Define the range for the curve within the existing points
        x_min, x_max = min(x_points_unique), max(x_points_unique)

        # Generate x values for the curve within the specified range
        x_curve = np.linspace(x_min, x_max, 300)

        fig.add_trace(go.Scatter(
            x=df_time_lookup[pp[0]],
            y=np.array(df_time_lookup[pp[1]]- 720) * -1,
            mode='lines',
            line=dict(color=colors[index]),
            name=f'DOWN Measure {measure_idx} - Angle {index}'
        ),
            row=1, col=1)
        fig.update_layout(yaxis_range=[0,1080])
        fig.update_layout(xaxis_range=[0,720])
        agregated_points_down_measure_x.append(df_time_lookup[pp[0]])
        agregated_points_down_measure_y.append(df_time_lookup[pp[1]])
    agregated_points_down_x.append(agregated_points_down_measure_x)
    agregated_points_down_y.append(agregated_points_down_measure_y)


    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] >interva[1]) & (df_raw_angles["time"] < interva[2])]
    print(interva[2]-interva[0])
    for index in range(0, len(trajectory_points_name)):
        # pp = trajectory_points_name[index]
        pp = trajectory_points_name[index]

        unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]

        # Create a cubic spline interpolation
        x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
        y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1
        spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

        # Define the range for the curve within the existing points
        x_min, x_max = min(x_points_unique), max(x_points_unique)

        # Generate x values for the curve within the specified range
        x_curve = np.linspace(x_min, x_max, 30)
        fig.add_trace(go.Scatter(
            x=df_time_lookup[pp[0]],
            y=np.array(df_time_lookup[pp[1]]- 720) * -1,
            mode='lines',
            line=dict(color=colors[index]),
            name=f'UP Measure {measure_idx} - Angle {index}'

        ),
            row=2, col=1)
        fig.update_layout()
        fig.update_layout()


# Add labels and legend
fig.update_layout(
    # xaxis=dict(title='X-axis', range=[0, 1080]),
    # yaxis=dict(title='Y-axis', range=[0, 720]),
    # legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=1000,
    width=1000,
    yaxis_range=[0,1080],
    xaxis_range=[0,720]
)


# Show the plot
fig.show()



# # Sample data
# angle = 2
#
# # Convert lists to numpy arrays
# x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
# y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1
#
# # Define the window size
# window_size = 50  # Adjust the window size as needed
#
# # Create a pandas DataFrame for easy rolling window operations
# df = pd.DataFrame({'x': x, 'y': y})
#
# # Sort DataFrame by 'x'
# df = df.sort_values(by='x')
#
# # Use a rolling window to calculate the weighted average
# def weighted_average(values, weights):
#     return np.sum(values * weights) / np.sum(weights)
#
# weighted_averages = []
#
# for i in range(len(df)):
#     start = max(0, i - window_size // 2)
#     end = min(len(df), i + window_size // 2 + 1)
#
#     weights = np.arange(1, end - start + 1)
#     weighted_averages.append(weighted_average(df['y'][start:end].values, weights))
#
# # Add the weighted averages to the DataFrame
# df['weighted_average'] = weighted_averages
#
# # Convert the DataFrame back to a dictionary
# average_data = dict(zip(df['x'], df['weighted_average']))
#
# x_avg = []
# y_avg = []
# for x, average_y in average_data.items():
#     x_avg.append(x)
#     y_avg.append(average_y)
#
# # # Plot the points and the curve
# # fig.add_trace(go.Scatter(
# #     x=x_avg,
# #     y=y_avg,
# #     mode='markers',
# #     marker=dict(color="black"),
# #     name=f'AVG - Angle {index}'
# # ))
#
# fig.add_trace(go.Scatter(
#     x=x_avg,
#     y=y_avg,
#     mode='lines',
#     line=dict(color="black"),
#     name=f'UP Measure {measure_idx} - Angle {index}'
# ))








In [ ]:
import numpy as np
# import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import interp1d

# Assuming 'loop_intervals' is defined before this code block
# loop_intervals = [(start_time_1, end_time_1, middle_time_1), (start_time_2, end_time_2, middle_time_2)]

# Initialize the figure with subplots, sharing the x-axis
# fig = make_subplots(rows=1, cols=2, subplot_titles=['First Interval', 'Second Interval'], shared_yaxes=True)

# Define the trajectory points and subplot colors
trajectory_points_name = [("i_p_0_x", "i_p_0_y"), ("i_p_1_x", "i_p_1_y"), ("i_p_2_x", "i_p_2_y"), ("i_p_3_x", "i_p_3_y")]
colors = ["blue", "pink", "green", "orange"]

# Lists to store aggregated points for each subplot
aggregated_points_down_x = []
aggregated_points_down_y = []
fig1 = go.Figure()
# Loop through each interval
for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[0]) & (df_raw_angles["time"] < interval[1])]

    # Lists to store individual measure points
    aggregated_points_down_measure_x = []
    aggregated_points_down_measure_y = []

    # Iterate through trajectory points
    for index in range(len(trajectory_points_name)):
        pp = trajectory_points_name[index]

        # Extract unique indices for data points
        unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]

        # Create a cubic spline interpolation
        x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
        y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1
        spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

        # Define the range for the curve within the existing points
        x_min, x_max = min(x_points_unique), max(x_points_unique)

        # Generate x values for the curve within the specified range
        x_curve = np.linspace(x_min, x_max, 300)

        # Add trace for the curve
        fig1.add_trace(go.Scatter(
            x=df_time_lookup[pp[0]],
            y=np.array(df_time_lookup[pp[1]] - 720) * -1,
            mode='lines',
            line=dict(color=colors[index]),
            name=f'DOWN Measure {measure_idx} - Angle {index}'
        ))


        # Append individual measure points to the lists
        aggregated_points_down_measure_x.append(df_time_lookup[pp[0]])
        aggregated_points_down_measure_y.append(df_time_lookup[pp[1]])

    # Append aggregated measure points to the lists
    aggregated_points_down_x.append(aggregated_points_down_measure_x)
    aggregated_points_down_y.append(aggregated_points_down_measure_y)



# # Update the overall layout with labels, legend, and dimensions
fig1.update_layout(
    height=500,
    width=1000,
    xaxis_range=[600,1000],
    yaxis_range=[300,700]

)

# Show the plot
fig1.show()


In [ ]:
 # Extract data within the next interval
fig2 = go.Figure()

for measure_idx, interval in enumerate(loop_intervals):
    # Extract data within the interval
    df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] > interval[1]) & (df_raw_angles["time"] < interval[2])]

    # Iterate through trajectory points for the second subplot
    for index in range(len(trajectory_points_name)):
        pp = trajectory_points_name[index]

        # Extract unique indices for data points
        unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]

        # Create a cubic spline interpolation
        x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
        y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1
        spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

        # Define the range for the curve within the existing points
        x_min, x_max = min(x_points_unique), max(x_points_unique)

        # Generate x values for the curve within the specified range
        x_curve = np.linspace(x_min, x_max, 30)

        # Add trace for the curve in the second subplot
        fig2.add_trace(go.Scatter(
            x=df_time_lookup[pp[0]],
            y=np.array(df_time_lookup[pp[1]] - 720) * -1,
            mode='lines',
            line=dict(color=colors[index]),
            name=f'UP Measure {measure_idx} - Angle {index}'
        ))

# # Update the overall layout with labels, legend, and dimensions
fig2.update_layout(
    height=500,
    width=1000,
    xaxis_range=[600,1000],
    yaxis_range=[300,700]
)
# Show the plot
fig2.show()

In [ ]:
# Show the plot
fig1.show()

In [ ]:
# Show the plot
fig2.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d
import pandas as pd

fig = go.Figure()

# Define constants
degree = 1  # You can adjust the degree of the polynomial
trajectory_points_name = [("i_p_0_x", "i_p_0_y"), ("i_p_1_x", "i_p_1_y"), ("i_p_2_x", "i_p_2_y"), ("i_p_3_x", "i_p_3_y")]
colors = ["blue", "pink", "green", "orange"]
window_size = 30  # Adjust the window size as needed

# Function to calculate rolling average for a given trajectory
def calculate_rolling_average(df, trajectory_name):
    df_trajectory = df[trajectory_name]
    df_trajectory['rolling_average'] = df_trajectory['y'].rolling(window=window_size, center=True).mean()
    df_trajectory = df_trajectory.dropna()
    return df_trajectory['x'].values, df_trajectory['rolling_average'].values

# Loop through angles 1, 2, and 3
for angle in [1, 2, 3]:
    # Data collection
    x_values = []
    y_values = []

    # Loop through loop intervals
    for measure_idx, interva in enumerate(loop_intervals):
        for index, pp in enumerate(trajectory_points_name):
            df_time_lookup = df_raw_angles.loc[
                (df_raw_angles["time"] > interva[index]) & (df_raw_angles["time"] < interva[index + 1])]

            unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]
            x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
            y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1

            fig.add_trace(go.Scatter(
                x=x_points_unique,
                y=y_points_unique,
                mode='lines',
                line=dict(color=colors[index]),
                name=f'DOWN Measure {measure_idx} - Angle {angle}'
            ))

            x_values.extend(x_points_unique)
            y_values.extend(y_points_unique)

    # Calculate and plot the rolling average
    df_trajectory = pd.DataFrame({'x': x_values, 'y': y_values})
    x_avg, y_avg = calculate_rolling_average(df_trajectory, f'angle_{angle}')

    fig.add_trace(go.Scatter(
        x=x_avg,
        y=y_avg,
        mode='lines',
        line=dict(color="black"),
        name=f'Rolling Average - Angle {angle}'
    ))

# Add labels and legend
fig.update_layout(
    xaxis=dict(title='X-axis', range=[0, 1080]),
    yaxis=dict(title='Y-axis', range=[0, 720]),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600,
    width=1000
)

# Show the plot
fig.show()


In [ ]:

# Sample data
x_coordinates = x
y_coordinates = y

# Convert lists to numpy arrays
x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1

# Define the window size
window_size = 3  # Adjust the window size as needed

# Create a pandas DataFrame for easy rolling window operations
df = pd.DataFrame({'x': x, 'y': y})

# Sort DataFrame by 'x'
df = df.sort_values(by='x')

# Use a rolling window to calculate the average
df['rolling_average'] = df['y'].rolling(window=window_size, center=True).mean()

# Drop NaN values resulting from the rolling window
df = df.dropna()

# Convert the DataFrame back to a dictionary
average_data = dict(zip(df['x'], df['rolling_average']))

# Print the result
print("X-coordinate\tAverage Y-coordinate")
for x, average_y in average_data.items():
    print(f"{x}\t\t{average_y}")

In [ ]:
average_data

In [ ]:
# Sample data - 5 trajectories
trajectories = [
    {'x': [1, 2, 3], 'y': [10, 20, 30]},
    {'x': [1, 2, 3], 'y': [15, 25, 35]},
    {'x': [1, 2, 3], 'y': [12, 22, 32]},
    {'x': [1, 2, 3], 'y': [18, 28, 38]},
    {'x': [1, 2, 3], 'y': [14, 24, 34]}
]

# [agregated_points_down_x[0][angle]
trajectories = [
    {'x': agregated_points_down_x[0][angle], 'y': agregated_points_down_y[0][angle]},
    {'x': agregated_points_down_x[1][angle], 'y': agregated_points_down_y[1][angle]}

]

# Convert data to a pandas DataFrame
df_list = []
for i, trajectory in enumerate(trajectories):
    df = pd.DataFrame({'x': trajectory['x'], f'y_traj_{i+1}': trajectory['y']})
    df_list.append(df)

# Merge DataFrames on 'x'
merged_df = pd.concat(df_list, axis=1)

# Interpolate missing values
interpolated_df = merged_df.interpolate(method='linear', axis=1)

# Calculate the average trajectory
average_df = interpolated_df.mean(axis=1)

# Create traces for each trajectory
traces = []
for col in interpolated_df.columns[1:]:
    trace = go.Scatter(x=interpolated_df['x'], y=interpolated_df[col], mode='lines+markers', name=col)
    traces.append(trace)

# Create trace for the average trajectory
average_trace = go.Scatter(x=interpolated_df['x'], y=average_df, mode='lines+markers', name='Average Trajectory')

# Create layout
layout = go.Layout(title='Trajectories and Average', xaxis=dict(title='X-coordinate'), yaxis=dict(title='Average Y-coordinate'))

# Create figure and plot
fig = go.Figure(data=traces + [average_trace], layout=layout)
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Sample data (replace this with your DataFrame)
data = {
    'x_trajectory_1': [1, 2, 3],
    'y_trajectory_1': [2, 3, 5],
    'x_trajectory_2': [1, 2, 3],
    'y_trajectory_2': [1, 2, 4],
    # Add more columns as needed
}

# df = pd.DataFrame(data)

# Specify column names for x and y coordinates
x_columns = ['i_p_2_x', 'i_p_3_x']  # Add more columns as needed
y_columns = ['i_p_2_y', 'i_p_3_y']  # Add more columns as needed

# Extract x and y coordinates
x_values = df_time_lookup[x_columns].values.flatten()
y_values = df_time_lookup[y_columns].values.flatten()

# Calculate maximum, minimum, and average boundaries
x_min, y_min = np.min(x_values), np.min(y_values)
x_max, y_max = np.max(x_values), np.max(y_values)
x_avg, y_avg = np.mean(x_values), np.mean(y_values)

# Visualize trajectories using Plotly
fig = go.Figure()

# Add scatter plot for each trajectory
for i in range(len(x_columns)):
    x_traj, y_traj = df_time_lookup[x_columns[i]], df_time_lookup[y_columns[i]]
    fig.add_trace(go.Scatter(x=x_traj, y=y_traj, mode='lines', line=dict(width=2)))

# Add markers for the start and end points of each trajectory
for i in range(len(x_columns)):
    start_point = df_time_lookup.loc[df_time_lookup.index[0], [x_columns[i], y_columns[i]]]
    end_point = df_time_lookup.loc[df_time_lookup.index[-1], [x_columns[i], y_columns[i]]]
    fig.add_trace(go.Scatter(x=[start_point[0], end_point[0]], y=[start_point[1], end_point[1]],
                             mode='markers', marker=dict(size=8)))

# Add boundaries to the plot
fig.add_trace(go.Scatter(x=[x_min, x_max, x_avg], y=[y_min, y_max, y_avg],
                         mode='markers', marker=dict(size=10, color='red')))

# Customize layout
fig.update_layout(title='Trajectories and Boundaries',
                  xaxis_title='X-axis', yaxis_title='Y-axis',
                  showlegend=False)

# Show the plot
fig.show()


In [ ]:
time_start = 7.35
time_middle = 7.6
time_end = 7.8

In [ ]:
df_time_lookup.columns


In [ ]:
df_time_lookup = df_raw_angles.loc[(df_raw_angles["time"] >loop_intervals[1][0]) & (df_raw_angles["time"] < loop_intervals[1][1])]

In [ ]:
df_time_lookup.shape

In [ ]:

matrix_of_points_trajectory = []
for point_name in trajectory_points_name:
    points = list(zip(df_time_lookup[point_name[0]], df_time_lookup[point_name[1]]))
    # Find the minimum and maximum X and Y coordinates
    # min_x = min(df_angle_filter["i_p_1_x"])
    # max_x = max(df_angle_filter["i_p_1_x"])
    # min_y = min(df_angle_filter["i_p_1_y"])
    # max_y = max(df_angle_filter["i_p_1_y"])
    # print(min_x,max_x,min_y,max_y)

    # Create a matrix with zeros
    matrix = np.zeros((720,1080), dtype=int)

    # Fill the matrix with the points
    for x, y in points:
        matrix[y , x] = 1  # Adjust indices to fit within the matrix

    matrix_of_points_trajectory.append(matrix)



In [ ]:
plt.figure(figsize= (15, 10))
plt.rcParams['font.size']=18
plt.subplot(221)
plt.imshow(matrix_of_points_trajectory[0]>0)
plt.subplot(222)
plt.imshow(matrix_of_points_trajectory[1]>0)
# plt.colorbar()
plt.title('L')
plt.subplot(223)
plt.imshow(matrix_of_points_trajectory[2]>0)
# plt.colorbar()
plt.title('a')
plt.subplot(224)
plt.imshow(matrix_of_points_trajectory[3]>0)
# plt.colorbar()
plt.title('b')

In [ ]:
# Create a curve using numpy's polyfit function
degree = 4  # You can adjust the degree of the polynomial
from scipy.interpolate import interp1d
colorss = ["blue", "pink", "green", "orange"]
plt.figure(figsize=(20,10))


for index in range(0,len(trajectory_points_name)):
    pp = trajectory_points_name[index]

    unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]
    # Create a cubic spline interpolation

    x_points_unique = (np.array(df_time_lookup[pp[0]])[unique_indices])
    y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices]- 720) * -1

    # Create a cubic spline interpolation
    spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

    # Define the range for the curve within the existing points
    x_min, x_max = min(x_points_unique), max(x_points_unique)

    # Generate x values for the curve within the specified range
    x_curve = np.linspace(x_min, x_max, 3)



    # Plot the points and the curve
    plt.scatter(x_points_unique, y_points_unique, label='Points', color=colorss[index])
    # plt.plot(x_curve, spline_interp(x_curve), label='Cubic Spline', color='red')

# Add labels and legend
plt.xlim(0, 1080)
plt.ylim(0, 720)
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# fit a second degree polynomial to the economic data
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot

# define the true objective function
angle = 1
def objective(x, a, b, c):
    return a * x + b * x**2 + c

# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/longley.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
# choose the input and output variables
x, y = x_points_unique, y_points_unique

x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c = popt
print('y = %.5f * x + %.5f * x^2 + %.5f' % (a, b, c))
# plot input vs output
pyplot.scatter(x, y)
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c)
# create a line plot for the mapping function
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

In [ ]:
x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1

In [ ]:
x

In [ ]:
# fit a second degree polynomial to the economic data
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot

# define the true objective function
angle = 2
def objective(x, a, b, c):
    return a * x + b * x**2 + c

# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/longley.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
# choose the input and output variables
x, y = x_points_unique, y_points_unique

x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c = popt
print('y = %.5f * x + %.5f * x^2 + %.5f' % (a, b, c))
# plot input vs output
pyplot.scatter(x, y)
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c)
# create a line plot for the mapping function
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

In [ ]:
# fit a second degree polynomial to the economic data
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot

# define the true objective function
angle = 3
def objective(x, a, b, c):
    return a * x + b * x**2 + c

# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/longley.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
# choose the input and output variables
x, y = x_points_unique, y_points_unique

x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c = popt
print('y = %.5f * x + %.5f * x^2 + %.5f' % (a, b, c))
# plot input vs output
pyplot.scatter(x, y)
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c)
# create a line plot for the mapping function
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d

# Create a curve using numpy's polyfit function
degree = 1  # You can adjust the degree of the polynomial

colors = ["blue", "pink", "green", "orange"]



fig = go.Figure()
for index in range(0, len(trajectory_points_name)):
    pp = trajectory_points_name[index]

    unique_indices = np.unique(df_time_lookup[pp[0]], return_index=True)[1]

    # Create a cubic spline interpolation
    x_points_unique = np.array(df_time_lookup[pp[0]])[unique_indices]
    y_points_unique = (np.array(df_time_lookup[pp[1]])[unique_indices] - 720) * -1
    spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

    # Define the range for the curve within the existing points
    x_min, x_max = min(x_points_unique), max(x_points_unique)

    # Generate x values for the curve within the specified range
    x_curve = np.linspace(x_min, x_max, 300)

    # Plot the points and the curve
    # fig.add_trace(go.Scatter(
    #     x=x_points_unique,
    #     y=y_points_unique,
    #     mode='markers',
    #     marker=dict(color=colors[index]),
    #     name=f'Points {index}'
    # ))

    fig.add_trace(go.Scatter(
        x=x_curve,
        y=spline_interp(x_curve),
        mode='lines',
        line=dict(color=colors[index]),
        name=f'Cubic Spline {index}'
    ))

# Add labels and legend
fig.update_layout(
    xaxis=dict(title='X-axis', range=[0, 1080]),
    yaxis=dict(title='Y-axis', range=[0, 720]),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600,
    width=1000
)

# Show the plot
fig.show()


In [ ]:
# fit a fifth degree polynomial to the economic data
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot
angle =0
# define the true objective function
def objective(x, a, b, c, d, e, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + f

# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/longley.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
# choose the input and output variables
x = np.concatenate([agregated_points_down_x[0][angle], agregated_points_down_x[1][angle], agregated_points_down_x[2][angle], agregated_points_down_x[3][angle]])
y = (np.concatenate([agregated_points_down_y[0][angle], agregated_points_down_y[1][angle],agregated_points_down_y[2][angle], agregated_points_down_y[3][angle]]) - 720) * -1
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c, d, e, f = popt
# plot input vs output
pyplot.scatter(x, y)
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c, d, e, f)
# create a line plot for the mapping function
pyplot.plot(x_line, y_line, '--', color='red')
pyplot.show()

In [ ]:
# zobrat viacere useky a spriemerovat ich
# zobrazit to v plotly
df_time_lookup.shape

In [ ]:

df_time_lookup_view = df_time_lookup
col_list = np.array(["i_p_0_x", "i_p_0_y","i_p_1_x", "i_p_1_y","i_p_2_x", "i_p_2_y","i_p_3_x", "i_p_3_y"])

for col in col_list:
    print(col)
    df_time_lookup_view = replace_extremes_with_average(df_time_lookup_view,col,100)


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import interp1d

# Create a curve using numpy's polyfit function
degree = 1  # You can adjust the degree of the polynomial

colors = ["blue", "pink", "green", "orange"]



fig = go.Figure()
for index in range(0, len(trajectory_points_name)):
    pp = trajectory_points_name[index]

    unique_indices = np.unique(df_time_lookup_view[pp[0]], return_index=True)[1]

    # Create a cubic spline interpolation
    x_points_unique = np.array(df_time_lookup_view[pp[0]])[unique_indices]
    y_points_unique = (np.array(df_time_lookup_view[pp[1]])[unique_indices] - 720) * -1
    spline_interp = interp1d(x_points_unique, y_points_unique, kind='cubic')

    # Define the range for the curve within the existing points
    x_min, x_max = min(x_points_unique), max(x_points_unique)

    # Generate x values for the curve within the specified range
    x_curve = np.linspace(x_min, x_max, 100)

    # Plot the points and the curve
    fig.add_trace(go.Scatter(
        x=x_points_unique,
        y=y_points_unique,
        mode='markers',
        marker=dict(color=colors[index]),
        name=f'Points {index}'
    ))

    fig.add_trace(go.Scatter(
        x=x_curve,
        y=spline_interp(x_curve),
        mode='lines',
        line=dict(color='red'),
        name=f'Cubic Spline {index}'
    ))

# Add labels and legend
fig.update_layout(
    xaxis=dict(title='X-axis', range=[0, 1080]),
    yaxis=dict(title='Y-axis', range=[0, 720]),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600,
    width=1000
)

# Show the plot
fig.show()


In [ ]:
df_raw_angles["positive_move_mask"]

In [ ]:
df_raw_angles["positive_move_mask"].sum()

In [ ]:
np.all(
    np.array([df_raw_angles["angle_0"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0,
     df_raw_angles["angle_1"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0,
     df_raw_angles["angle_1"].rolling(window,center = True).apply(lambda x: sum(sorted(x)[1:window-1])) > 0] ),axis=0)

In [ ]:
df_raw_angles["angle_0"].rolling(10).apply(lambda x: np.mean(sorted(x)[1:10-1]))

In [ ]:
df_raw_angles["angle_0"]